In [133]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

### Loading the data
---

fetch the data

In [134]:
wb_data = {
    "dwarf" : open("data/worldbox/dwarf").read().splitlines(),
    "elf"   :   open("data/worldbox/elf").read().splitlines(),
    "orc"   :   open("data/worldbox/orc").read().splitlines()
}

xn_data = {
    "dwarf"      : list(set(open("data/xanathar/dwarf_male.txt").read().splitlines()    + open("data/xanathar/dwarf_female.txt").read().splitlines()    + open("data/fantasynamegen/dwarf").read().splitlines())),
    "elf"        : list(set(open("data/xanathar/elf_male.txt").read().splitlines()      + open("data/xanathar/elf_female.txt").read().splitlines()      + open("data/fantasynamegen/elf").read().splitlines())),
    "orc"        : list(set(open("data/xanathar/halforc_male.txt").read().splitlines()  + open("data/xanathar/halforc_female.txt").read().splitlines()  + open("data/fantasynamegen/orc").read().splitlines())),
    "tiefling"   : list(set(open("data/xanathar/tiefling_male.txt").read().splitlines() + open("data/xanathar/tiefling_female.txt").read().splitlines() + open("data/fantasynamegen/tiefling").read().splitlines())),
    "halfling"   : list(set(open("data/xanathar/halfling_male.txt").read().splitlines() + open("data/xanathar/halfling_female.txt").read().splitlines() + open("data/fantasynamegen/halfling").read().splitlines())),
    "dragonborn" : list(set(open("data/xanathar/halfling_male.txt").read().splitlines() + open("data/xanathar/halfling_female.txt").read().splitlines() + open("data/fantasynamegen/dragonborn").read().splitlines()))
}


print("total worldbox data : ", sum(len(d) for d in wb_data.values()))
print("max worldbox data : ", max(len(p) for d in wb_data.values() for p in d))


total worldbox data :  4412
max worldbox data :  12


clean data

In [135]:
for i in xn_data.keys():
    xn_data[i] = list(set(d.strip().lower() for d in xn_data[i]))


stoi = { s:i for i,s in enumerate(list(p for d in xn_data.values() for p in d)) }
itos = { i:s for s,i in stoi.items()}

label_stoi = {s:i for i,s in enumerate(list(xn_data.keys()))}
label_itos = {s:i for i,s in label_stoi.items()}


# print(len(set(p for d in xn_data.values() for p in d)))

my_dict = {i:list(p for d in xn_data.values() for p in d).count(i) for i in list(p for d in xn_data.values() for p in d)   }


print(len(list(stoi.items())))
print("total xanathar data : ", sum(len(d) for d in xn_data.values()) )


6184
total xanathar data :  6329


In [136]:
label_stoi

{'dwarf': 0, 'elf': 1, 'orc': 2, 'tiefling': 3, 'halfling': 4, 'dragonborn': 5}

In [137]:
max(len(p) for d in xn_data.values() for p in d for c in p)

18

In [138]:
def charsplit(s:str, pad=18):
    return [ord(d)-ord('a')+1.0 for d in s] + ([0] * (pad-len(s)))

charsplit("abcdefg")

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [139]:

Xtr, Ytr = list(charsplit(p) for d in xn_data.values() for p in d), list(label_stoi[p] for d in xn_data.keys() for p in [d]*len(xn_data[d]))
Xtr = torch.tensor(Xtr)
Ytr = torch.tensor(Ytr)
len(Xtr), len(Ytr)

(6329, 6329)

In [140]:
# only with no stoi[p] and label_stoi[p] in Xtr,Ytr definition
# check = {"dwarf":[], "elf":[], "orc":[], "tiefling":[], "halfling":[]}
# for i in zip(Ytr,Xtr):
#     check[i[0]].append(i[1])
# 
# #xn_data == check
# print(Xtr[191], Ytr[440])
# 
# for i in range(len(Xtr)):
#     if Xtr[i] == 'nora':
#         print(i)

Xtr.dtype

torch.float32

### Making the model
---

In [141]:

# C = torch.randn((26, 10))
# 
# model = torch.nn.Sequential(
#     torch.nn.Linear(12 , 100),    
#     torch.nn.Linear(100,  50),   torch.nn.Tanh(),
#     torch.nn.Linear(50 , 25) ,   torch.nn.Tanh(),
#     torch.nn.Linear(25 , 5) 
# )
# 
# for n in model.children():
#     n.requires_grad = True
# 
# parameters = model.parameters()
# sum(p.nelement() for p in parameters)

In [142]:
W1 = torch.randn((18, 100))
b1 = torch.randn(100)
W2 = torch.randn((100,50))
b2 = torch.randn(50)
W3 = torch.randn((50,6))
b3 = torch.randn(6)

parameters = [W1,b1,W2,b2,W3,b3]
for p in parameters:
    p.requires_grad = True

In [148]:
batch_size = 32
epochs = 1000

for i in range(epochs):
    ix = torch.randint(0, 6329, (batch_size, ))
    Xb, Yb = Xtr[ix], Ytr[ix]

    #emb = 
    #X = torch.randn((32,12))dk
    l1 = torch.tanh(Xb @ W1 + b1)
    l2 = torch.tanh(l1 @ W2 + b2)
    logits = l2 @ W3 + b3

    loss = F.cross_entropy(logits, Yb)

    if i % 100 == 0:
        print(loss.item())

    for i in parameters:
        i.grad = None

    loss.backward()

    for i in parameters:
        i.data += -0.001 * i.grad 



0.9851483702659607
1.3117640018463135
0.8021224141120911
1.2573732137680054
1.1338024139404297
1.0309337377548218
1.0504716634750366
1.22589111328125
1.1100701093673706
1.2352482080459595


In [165]:
prompt = ""
prompt = charsplit(prompt)
prompt = torch.tensor(prompt)


l1 = torch.tanh(prompt @ W1 + b1)
l2 = torch.tanh(l1 @ W2 + b2)
logits = l2 @ W3 + b3
probs = F.softmax(logits, dim=0)
print(probs.data)

tensor([0.3093, 0.0102, 0.5261, 0.0075, 0.0012, 0.1457])


In [114]:
label_stoi

{'dwarf': 0, 'elf': 1, 'orc': 2, 'tiefling': 3, 'halfling': 4, 'dragonborn': 5}

SyntaxError: unterminated string literal (detected at line 2) (589144943.py, line 2)